**IMPORTING BASIC LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

**READING THE DATASET**

In [2]:
df=pd.read_csv("data/CTG.csv")


**CLEANING THE DATA AND PREPARING INPUT AND OUTPUT FIELDS**

In [3]:
df=df.drop(["FileName","Date","SegFile","b","e"],axis=1)
df=df.dropna()
X=df[['LBE', 'LB', 'AC', 'FM', 'UC', 'DL',
       'DS', 'DP', 'DR']]
Y=df[["NSP"]]

**PERFORMING THE SCALING**

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
Scaler=StandardScaler()
X=Scaler.fit_transform(X)

**SPLITTING THE DATASET INTO TRAINING SET AND TEST SET**

In [5]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

**SUPPORT VECTOR CLASSIFIER MODEL**

In [6]:
from sklearn.svm import SVC

svm_clf=SVC(kernel="poly",degree=6,coef0=5,gamma=0.1)
svm_clf=svm_clf.fit(X_train,y_train)
y_pred=svm_clf.predict(X_test)

**CALCULATING DIFFERENT METRICS**

In [10]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
print("CONFUSION MATRIX")
print(confusion_matrix(y_test,y_pred))
print("F1-score:",f1_score(y_test,y_pred,average='weighted'))
print("Accuracy-score:",accuracy_score(y_test,y_pred))
print("Precision-score:",precision_score(y_test,y_pred,average='weighted'))
print("Recall-score:",recall_score(y_test,y_pred,average="weighted"))

CONFUSION MATRIX
[[470  20   6]
 [ 29  62  10]
 [  8   5  28]]
F1-score: 0.8753514883881671
Accuracy-score: 0.877742946708464
Precision-score: 0.8744054575648664
Recall-score: 0.877742946708464


In [13]:
import joblib
joblib.dump(svm_clf, './models/svc-model')
#model=joblib.load('./models/svc-model') for loading a model

['./models/svc-model']